In [0]:
from pyspark.sql.types import StructType, StructField, StringType

schema = StructType([
    StructField('id', StringType(), True),
    StructField('presentation', StructType([
        StructField('title', StringType(), True),
        StructField('suggestionTitle', StringType(), True),
        StructField('subtitle', StringType(), True)
    ]), True),
    StructField('navigation', StructType([
        StructField('entityId', StringType(), True),
        StructField('entityType', StringType(), True),
        StructField('localizedName', StringType(), True),
        StructField('relevantFlightParams', StructType([
            StructField('skyId', StringType(), True),
            StructField('entityId', StringType(), True),
            StructField('flightPlaceType', StringType(), True),
            StructField('localizedName', StringType(), True)
        ]), True),
        StructField('relevantHotelParams', StructType([
            StructField('entityId', StringType(), True),
            StructField('entityType', StringType(), True),
            StructField('localizedName', StringType(), True)
        ]), True)
    ]), True)
])


In [0]:
import requests
import json
from pyspark.sql.functions import *
headers = {
    "x-rapidapi-host": "skyscanner80.p.rapidapi.com",
    "x-rapidapi-key": "da5e4cb143mshb0961ab3efe73aep1f7bb7jsnd5bbdf38c6dc"  
}
df = spark.read.table('Country')

response = requests.get('https://skyscanner80.p.rapidapi.com/api/v1/flights/auto-complete?query=new&market=US&locale=en-US', headers=headers)
# Check if the request was successful
if response.status_code == 200:
    # Parse JSON response
   airport=response.json()['data']
   rdd = spark.sparkContext.parallelize(countrydata)
   airportdf= spark.createDataFrame(rdd,schema=schema)
else:
    print(f"Error: {response.status_code}, {response.text}")


In [0]:
df=airportdf.withColumn("entityId",col("navigation.entityId"))\
    .withColumn("flightPlaceType",col("navigation.relevantFlightParams.flightPlaceType"))\
    .withColumn("localizedName",col("navigation.relevantFlightParams.localizedName"))\
    .withColumn("skyId",col("navigation.relevantFlightParams.skyId"))\
    .withColumn("entityType",col("navigation.relevantHotelParams.entityType"))\
    .withColumn("localizedName",col("navigation.relevantHotelParams.localizedName"))\
    .withColumn("title",col("presentation.title"))\
    .withColumn("subtitle",col("presentation.subtitle"))\
    .drop('navigation')\
    .drop('presentation')
df.display()

id,entityId,flightPlaceType,localizedName,skyId,entityType,title,subtitle
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India
eyJzIjoiTkNMIiwiZSI6Ijk1Njc0MDQ0IiwiaCI6IjI3NTQ1MDkyIn0=,95674044,AIRPORT,Newcastle,NCL,CITY,Newcastle,United Kingdom
eyJzIjoiTlFZIiwiZSI6Ijk1NjczOTYzIiwiaCI6IjI3NTQ1MTQ5In0=,95673963,AIRPORT,Newquay,NQY,CITY,Newquay,United Kingdom
eyJzIjoiTllDQSIsImUiOiIyNzUzNzU0MiIsImgiOiIyNzUzNzU0MiJ9,27537542,CITY,New York,NYCA,CITY,New York,United States
eyJzIjoiRVdSIiwiZSI6Ijk1NTY1MDU5IiwiaCI6IjI3NTM3NTQyIn0=,95565059,AIRPORT,New York,EWR,CITY,New York Newark,United States
eyJzIjoiSkZLIiwiZSI6Ijk1NTY1MDU4IiwiaCI6IjI3NTM3NTQyIn0=,95565058,AIRPORT,New York,JFK,CITY,New York John F. Kennedy,United States
eyJzIjoiTEdBIiwiZSI6Ijk1NTY1MDU3IiwiaCI6IjI3NTM3NTQyIn0=,95565057,AIRPORT,New York,LGA,CITY,New York LaGuardia,United States
eyJzIjoiU1dGIiwiZSI6Ijk1NTY2MjgwIiwiaCI6IjI3NTM3NTQyIn0=,95566280,AIRPORT,New York,SWF,CITY,Stewart International,United States


In [0]:
df.write.mode('Overwrite').saveAsTable('Airport')

In [0]:
%sql
select * from  airport;

id,entityId,flightPlaceType,localizedName,skyId,entityType,title,subtitle
eyJzIjoiSkZLIiwiZSI6Ijk1NTY1MDU4IiwiaCI6IjI3NTM3NTQyIn0=,95565058,AIRPORT,New York,JFK,CITY,New York John F. Kennedy,United States
eyJzIjoiREVMIiwiZSI6Ijk1NjczNDk4IiwiaCI6IjI3NTQwNzA2In0=,95673498,AIRPORT,New Delhi,DEL,CITY,Indira Gandhi International,India
eyJzIjoiU1dGIiwiZSI6Ijk1NTY2MjgwIiwiaCI6IjI3NTM3NTQyIn0=,95566280,AIRPORT,New York,SWF,CITY,Stewart International,United States
eyJzIjoiTEdBIiwiZSI6Ijk1NTY1MDU3IiwiaCI6IjI3NTM3NTQyIn0=,95565057,AIRPORT,New York,LGA,CITY,New York LaGuardia,United States
eyJzIjoiRVdSIiwiZSI6Ijk1NTY1MDU5IiwiaCI6IjI3NTM3NTQyIn0=,95565059,AIRPORT,New York,EWR,CITY,New York Newark,United States
eyJzIjoiTkNMIiwiZSI6Ijk1Njc0MDQ0IiwiaCI6IjI3NTQ1MDkyIn0=,95674044,AIRPORT,Newcastle,NCL,CITY,Newcastle,United Kingdom
eyJzIjoiTlFZIiwiZSI6Ijk1NjczOTYzIiwiaCI6IjI3NTQ1MTQ5In0=,95673963,AIRPORT,Newquay,NQY,CITY,Newquay,United Kingdom
eyJzIjoiTllDQSIsImUiOiIyNzUzNzU0MiIsImgiOiIyNzUzNzU0MiJ9,27537542,CITY,New York,NYCA,CITY,New York,United States
